In [3]:
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

Using TensorFlow backend.


In [6]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from subprocess  import check_output
print(check_output(["ls"]).decode("utf8"))

Comparing Old data results.ipynb
DL_FF_RL _Data.ipynb
FF_keras.ipynb
FetalFraction_with_DL-RL data_tested.ipynb
FetalFraction_with_DL_GC.ipynb
FetalFraction_with_DL_GC_tested.ipynb
Final_WIth_High_Prediction_GC.ipynb
Final_WIth_High_Prediction_RL.ipynb
HousePrice2019May30_A1822 Github.ipynb
Only_Ridge_Lasso_Enet_bayesian_ridge_GC.ipynb
Only_Ridge_Lasso_Enet_bayesian_ridge_RL_Function.ipynb
Regression_with_Keras.ipynb
Tensor_flow_basic_jmodel.ipynb
housingdata.csv
ridge_pred.csv



In [7]:
data = pd.read_csv("housingdata.csv",header = None)

In [9]:
data.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.9,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.9,9.14,21.6


In [11]:
# y is 13 Col and remainings are x
x = data.iloc[:,0:13]
y = data.iloc[:,13]

In [12]:
x.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.9,4.98
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.9,9.14


In [13]:
y.head(2)

0    24.0
1    21.6
Name: 13, dtype: float64

In [15]:
x.shape,y.shape

((506, 13), (506,))

## 2. Develope a Baseline Neural Network Model

In [16]:
# Define base Model
def baseline_model():
    # Create Model
    model = Sequential()
    model.add(Dense(13,input_dim= 13,kernel_initializer = 'normal',activation = "relu"))
    model.add(Dense(1,kernel_initializer = "normal"))
    
    #Compile model
    model.compile(loss = "mean_squared_error", optimizer = 'adam')
    return model

In [17]:
# Fix random seed for reprocucibility 
seed  = 7
np.random.seed(seed)

# Evaluate model with standardize dataset
estimator = KerasRegressor(build_fn=baseline_model, nb_epoch = 100, batch_size = 5, verbose = 0)

In [19]:
kfold = KFold(n_splits = 10, random_state = seed)
results = cross_val_score(estimator, x.values, y.values , cv = kfold, n_jobs= 1)
print("Results :%.2f (%.2f)MSE"%(results.mean(),results.std()))

Results :-114.42 (82.48)MSE


## 3. Modeling The Standardized Dataset

In [21]:
np.random.seed(seed)
estimators = []
estimators.append(("standardize", StandardScaler()))
estimators.append(('mlp',KerasRegressor(build_fn = baseline_model, epochs = 50, batch_size =5, verbose =0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10 , random_state = seed)
results = cross_val_score(pipeline, x,y,cv = kfold, n_jobs = 1)
print("Standardized: %.2f (%.2f)MSE"%(results.mean(), results.std()))

Standardized: -29.20 (27.52)MSE


## Evaluate a Deeper Network Topology

In [22]:
# define the model
def larger_model():
    # Create model
    model = Sequential()
    model.add(Dense(13,input_dim = 13 , kernel_initializer = 'normal', activation = 'relu'))
    model.add(Dense(6,kernel_initializer = "normal",activation = "relu"))
    model.add(Dense(1,kernel_initializer = 'normal'))
    
    # Compile Model
    model.compile(loss = "mean_squared_error", optimizer = "adam")
    return model

In [26]:
np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, x, y, cv=kfold, n_jobs=1)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: -23.38 (26.01) MSE


## 5. Evaluate a Wider Network Topology

In [38]:
# define wider model
def wider_model():
    # create model
    model = Sequential()
    model.add(Dense(20, input_dim=13, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [37]:
np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=wider_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, x, y, cv=kfold, n_jobs=1)
print("Wider: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Wider: -23.41 (29.32) MSE
